In [1]:
import os
print(os.getcwd())

g:\DIYguru\Data-Science-and-Engineering-Analytics


In [2]:
'''
Created a function 'load_data_excel' to load excel into python
'''

import pandas as pd
import numpy as np
import os

def load_data_excel(file_path):
    """
    Load data from an Excel file and return a DataFrame.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    df = pd.read_excel(file_path)
    return df

In [ ]:
'''
Loaded VED_Static_Data_ICE&HEV into dataframe df_ICE_HEV using the above function
Loaded VED_Static_Data_PHEV&EV into dataframe df_PHEV_EV using the above function
'''

DATA_DIR = load_data_excel("G:\\DIYguru\\Notes and Sample Data\\VED-master\\Data\\VED_Static_Data_ICE&HEV.xlsx")



In [3]:
import os
import pandas as pd

def load_files_from_directory(directory_path, file_types=('csv', 'xls', 'xlsx')):
    """
    Load and append CSV and Excel files from a given directory into a single pandas DataFrame.

    Args:
        directory_path (str): Path to the directory containing the files.
        file_types (tuple): File types to load. Default is ('csv', 'xls', 'xlsx').

    Returns:
        pd.DataFrame: A single DataFrame containing data from all files.
    """
    dataframes = []
    for filename in os.listdir(directory_path):
        filepath = os.path.join(directory_path, filename)
        if os.path.isfile(filepath):
            ext = filename.lower().split('.')[-1]
            try:
                if ext == 'csv' and 'csv' in file_types:
                    df = pd.read_csv(filepath)
                    dataframes.append(df)
                elif ext in ['xls', 'xlsx'] and ext in file_types:
                    df = pd.read_excel(filepath)
                    dataframes.append(df)
            except Exception as e:
                print(f"Failed to load {filename}: {e}")
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no files are loaded

In [4]:
DATA_DIR = r"G:\DIYguru\Notes and Sample Data\DEVRT\DEVRT\NISSAN LEAF"
DATA_DIR = load_files_from_directory(DATA_DIR)

print(f"Combined DataFrame shape: {DATA_DIR.shape}")

Combined DataFrame shape: (5843, 49)


In [5]:
'''
This cell loads AWS credentials from a .env file using python-dotenv,
creates a boto3 S3 client with those credentials,
and lists all S3 buckets in the account.
'''

import os
from dotenv import load_dotenv
import boto3

# Load environment variables from .env
load_dotenv()

# Create boto3 client using loaded environment variables
s3 = boto3.client("s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_DEFAULT_REGION")
)

# Example: list buckets
buckets = s3.list_buckets()
print("Your S3 Buckets:")
for bucket in buckets['Buckets']:
    print(f" - {bucket['Name']}")

Your S3 Buckets:
 - s3aravindh973515031797


In [10]:
bucket_name = 's3aravindh973515031797'

response = s3.list_objects_v2(Bucket=bucket_name)
for item in response.get("Contents", []):
    print(item["Key"])

Cleaned up VED Source Data/
Cleaned up VED Source Data/df_ICE_HEV.parquet
Cleaned up VED Source Data/df_PHEV_EV.parquet
Cleaned up VED Source Data/df_VED.parquet
Cleaned up VED Source Data/df_combined.parquet
Cleaned up VED Source Data/df_dynamic_sample.parquet
Cleaned up VED Source Data/df_static.parquet
DIYguru ML Source Data/
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171101_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171108_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171115_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171122_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171129_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171206_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171213_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171220_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_171227_week.csv
DIYguru ML Source Data/VED_DynamicData_Part1/VED_180103_week.csv
DIYguru ML Source D

In [7]:
def push_df_to_s3_parquet(df, object_key):
    """
    Pushes a pandas DataFrame to a predefined S3 bucket as a Parquet file.

    Args:
        df (pd.DataFrame): DataFrame to upload.
        object_key (str): S3 object key (path/filename.parquet).
    """
    from io import BytesIO
    parquet_buffer = BytesIO()
    df.to_parquet(parquet_buffer, index=False)
    parquet_buffer.seek(0)
    s3.upload_fileobj(parquet_buffer, bucket_name, object_key)
    print(f"DataFrame uploaded to s3://{bucket_name}/{object_key}")

# Example usage:
# push_df_to_s3_parquet(df_static, 'path/to/df_static.parquet')

In [8]:
push_df_to_s3_parquet(DATA_DIR, 'NISSAN LEAF/NISSAN_LEAF.parquet')

DataFrame uploaded to s3://s3aravindh973515031797/NISSAN LEAF/NISSAN_LEAF.parquet


In [16]:
from io import BytesIO

def read_parquet_from_s3(bucket_name, object_key):
        """
        Reads a Parquet file from an AWS S3 bucket using the global s3 client.

        Args:
            bucket_name: Name of the S3 bucket.
            object_key: Key (path) to the Parquet file in the S3 bucket.

        Returns:
            DataFrame containing the Parquet data.
        """
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        file_content = response['Body'].read()
        df = pd.read_parquet(BytesIO(file_content))
        return df

In [17]:
bucket_name = 's3aravindh973515031797'
df = 'NISSAN LEAF/NISSAN_LEAF.parquet'

In [18]:
df = read_parquet_from_s3(bucket_name, df)

In [21]:
df.head()

,timestamp_data_utc,elv_spy,speed,soc,amb_temp,soh,regenwh,Motor Pwr(w),Aux Pwr(100w),Motor Temp,...,cars_by_speed_interval_80_100,cars_by_speed_interval_100_120,cars_by_speed_interval_0_50,cars_by_speed_interval_50_80,cars_by_speed_interval_80_120,cars_by_speed_interval_120_inf,cars_by_length_interval_0_7,cars_by_length_interval_7_inf,max_speed,radius
0,18/04/2023 11:33,98,12.7,87,17.5,99.27,-617,1520,2,90,...,47.0,239.0,NaN,NaN,NaN,102.0,271.0,118.0,30,3.0
1,18/04/2023 11:33,98,32.1,87,17.5,99.27,-617,6200,2,89,...,47.0,239.0,NaN,NaN,NaN,102.0,271.0,118.0,30,3.0
2,18/04/2023 11:33,98,30.7,87,17.5,99.27,-617,9480,2,89,...,47.0,239.0,NaN,NaN,NaN,102.0,271.0,118.0,55,3.0
3,18/04/2023 11:33,98,34.4,87,17.5,99.27,-617,1960,2,89,...,47.0,239.0,NaN,NaN,NaN,102.0,271.0,118.0,25,3.0
4,18/04/2023 11:33,98,20.9,87,17.5,99.27,-626,0,2,88,...,47.0,239.0,NaN,NaN,NaN,102.0,271.0,118.0,90,3.0
